In [1]:
import spacy
import pandas as pd

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")



/Users/study/nlp_project/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
import fitz  # PyMuPDF

def read_pdf(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Example usage
pdf_text = read_pdf('resumes/sample_resume.pdf')  # Ensure the file name matches your actual file
print("PDF Text:", pdf_text[:500])  # Print first 500 characters for preview


PDF Text: INTERNSHIP REPORT ON 
Digital e sales dashboard 
By Sayan Sahu 
VT2024 
Under the guidance of Mr.Rahul Kumar 
About the Project 
Introduction 
The project aims to develop a comprehensive digital sales dashboard 
using Tableau to visualize and analyze sales data, enabling 
stakeholders to make data-driven decisions.
SCOPE
The scope of the project includes data collection, data preprocessing, 
dashboard design, and implementation using Tableau. The dashboard 
provides insights into key sales metri


In [3]:
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove extra spaces
    text = ' '.join(text.split())
    return text

# Preprocess the PDF text
processed_text = preprocess_text(pdf_text)
print("Processed Text:", processed_text[:500])  # Print first 500 characters for preview


Processed Text: internship report on digital e sales dashboard by sayan sahu vt2024 under the guidance of mr.rahul kumar about the project introduction the project aims to develop a comprehensive digital sales dashboard using tableau to visualize and analyze sales data, enabling stakeholders to make data-driven decisions. scope the scope of the project includes data collection, data preprocessing, dashboard design, and implementation using tableau. the dashboard provides insights into key sales metrics, trends,


In [4]:
import spacy

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

# Extract entities from the processed text
entities = extract_entities(processed_text)
print("Extracted Entities:", entities)



Extracted Entities: [('sayan sahu vt2024', 'PERSON'), ('mr.rahul kumar', 'PERSON'), ('1', 'CARDINAL'), ('monthly', 'DATE'), ('the past two years', 'DATE'), ('monthly', 'DATE'), ('months and years', 'DATE'), ('2', 'CARDINAL'), ('a month', 'DATE'), ('3', 'CARDINAL'), ('first', 'ORDINAL'), ('second', 'ORDINAL'), ('5', 'CARDINAL'), ('5', 'CARDINAL'), ('5', 'CARDINAL'), ('4', 'CARDINAL'), ('first', 'ORDINAL'), ('first', 'ORDINAL'), ('months and year', 'DATE'), ('second', 'ORDINAL'), ('month', 'DATE'), ('year', 'DATE'), ('5', 'CARDINAL'), ('south east asia', 'LOC'), ('6', 'CARDINAL'), ('north asia', 'LOC'), ('year and months', 'DATE'), ('7', 'CARDINAL'), ('red & green threshold', 'ORG'), ('8', 'CARDINAL'), ('9', 'CARDINAL'), ('3', 'CARDINAL'), ('4', 'CARDINAL'), ('5', 'CARDINAL'), ('months and years', 'DATE')]


In [5]:
# Define criteria for shortlisting
criteria = {
    'skills': ['tableau', 'data analysis'],
    'locations': ['south east asia', 'north asia'],
    'experience_years': ['2', '3', '5']
}

def check_criteria(entities, criteria):
    matches = {key: [] for key in criteria}
    
    for entity, label in entities:
        for key, values in criteria.items():
            if any(value.lower() in entity.lower() for value in values):
                matches[key].append(entity)
    
    return matches

# Check if the extracted entities match the defined criteria
matches = check_criteria(entities, criteria)
print("Matches with Criteria:", matches)


Matches with Criteria: {'skills': [], 'locations': ['south east asia', 'north asia'], 'experience_years': ['sayan sahu vt2024', '2', '3', '5', '5', '5', '5', '3', '5']}


In [6]:
def check_criteria_advanced(text, criteria):
    matches = {key: [] for key in criteria}
    
    text = text.lower()  # Convert entire text to lowercase for case-insensitive matching
    
    for key, values in criteria.items():
        for value in values:
            if value.lower() in text:
                matches[key].append(value)
    
    return matches

# Check if the full text matches the defined criteria
advanced_matches = check_criteria_advanced(processed_text, criteria)
print("Advanced Matches with Criteria:", advanced_matches)


Advanced Matches with Criteria: {'skills': ['tableau'], 'locations': ['south east asia', 'north asia'], 'experience_years': ['2', '3', '5']}


In [7]:
import os
import fitz

# Define function to read all PDF files in a directory
def read_all_pdfs(directory):
    pdf_texts = {}
    for file_name in os.listdir(directory):
        if file_name.endswith('.pdf'):
            file_path = os.path.join(directory, file_name)
            pdf_texts[file_name] = read_pdf(file_path)
    return pdf_texts

# Process all resumes in the 'resumes' folder
pdf_texts = read_all_pdfs('resumes')

# Check which resumes match the criteria
def process_all_resumes(pdf_texts, criteria):
    results = {}
    for file_name, text in pdf_texts.items():
        processed_text = preprocess_text(text)
        matches = check_criteria_advanced(processed_text, criteria)
        if any(matches.values()):  # Check if there are any matches
            results[file_name] = matches
    return results

# Get the results for all resumes
all_matches = process_all_resumes(pdf_texts, criteria)
print("Number of matching resumes:", len(all_matches))

# Save the results to a file
save_results(all_matches, 'nlp_project/matching_results.txt')
print("Results saved to 'nlp_project/matching_results.txt'")


Number of matching resumes: 1


NameError: name 'save_results' is not defined

In [ ]:
def save_results(results, file_path):
    with open(file_path, 'w') as f:
        for file_name, match_details in results.items():
            f.write(f"Resume: {file_name}\n")
            for category, items in match_details.items():
                f.write(f"  {category.capitalize()}:\n")
                for item in items:
                    f.write(f"   - {item}\n")
            f.write("\n")

# Save the results to a file again
save_results(all_matches, 'nlp_project/matching_results.txt')
print("Results saved to 'nlp_project/matching_results.txt'")


In [ ]:
def save_results(results, file_path):
    with open(file_path, 'w') as f:
        for file_name, match_details in results.items():
            f.write(f"Resume: {file_name}\n")
            for category, items in match_details.items():
                f.write(f"  {category.capitalize()}:\n")
                for item in items:
                    f.write(f"   - {item}\n")
            f.write("\n")

# Save the results to a file again
save_results(all_matches, 'nlp_project/matching_results.txt')
print("Results saved to 'nlp_project/matching_results.txt'")



In [ ]:
import os
import fitz

# Define function to read all PDF files in a directory
def read_all_pdfs(directory):
    pdf_texts = {}
    for file_name in os.listdir(directory):
        if file_name.endswith('.pdf'):
            file_path = os.path.join(directory, file_name)
            pdf_texts[file_name] = read_pdf(file_path)
    return pdf_texts

# Process all resumes in the 'resumes' folder
pdf_texts = read_all_pdfs('resumes')

# Check which resumes match the criteria
def process_all_resumes(pdf_texts, criteria):
    results = {}
    for file_name, text in pdf_texts.items():
        processed_text = preprocess_text(text)
        matches = check_criteria_advanced(processed_text, criteria)
        if any(matches.values()):  # Check if there are any matches
            results[file_name] = matches
    return results

# Get the results for all resumes
all_matches = process_all_resumes(pdf_texts, criteria)
print("Number of matching resumes:", len(all_matches))

# Save the results to a file
save_results(all_matches, 'nlp_project/matching_results.txt')
print("Results saved to 'nlp_project/matching_results.txt'")


In [ ]:
import os

# List contents of the current working directory
print("Contents of the current directory:")
print(os.listdir('.'))

# List contents of the 'nlp_project' directory to ensure it exists
print("\nContents of 'nlp_project' directory:")
print(os.listdir('nlp_project'))


In [ ]:
import os

# Re-create the directory
os.makedirs('nlp_project', exist_ok=True)

# Define the path for the results file
results_file_path = 'nlp_project/matching_results.txt'

# Save the results to the file
def save_results(results, file_path):
    with open(file_path, 'w') as f:
        for file_name, match_details in results.items():
            f.write(f"Resume: {file_name}\n")
            for category, items in match_details.items():
                f.write(f"  {category.capitalize()}:\n")
                for item in items:
                    f.write(f"   - {item}\n")
            f.write("\n")

save_results(all_matches, results_file_path)
print(f"Results saved to '{results_file_path}'")


In [ ]:
import os
import spacy
from PyPDF2 import PdfReader

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

def extract_text_from_pdf(file_path):
    text = ""
    with open(file_path, "rb") as file:
        reader = PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text

def process_resume(file_path):
    text = extract_text_from_pdf(file_path)
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

def match_criteria(entities):
    # Define your criteria here
    criteria = {
        'skills': ['tableau'],
        'locations': ['south east asia', 'north asia'],
        'experience_years': ['2', '3', '5']
    }
    
    matches = {'skills': [], 'locations': [], 'experience_years': []}
    
    # Check for skills
    for skill in criteria['skills']:
        if skill.lower() in ' '.join([e[0].lower() for e in entities]):
            matches['skills'].append(skill)
    
    # Check for locations
    for loc in criteria['locations']:
        if loc.lower() in ' '.join([e[0].lower() for e in entities]):
            matches['locations'].append(loc)
    
    # Check for experience years
    for exp in criteria['experience_years']:
        if exp in [e[0] for e in entities]:
            matches['experience_years'].append(exp)
    
    return matches

# Process all resumes in the directory
def process_all_resumes(directory):
    all_matches = {}
    for file_name in os.listdir(directory):
        if file_name.endswith(".pdf"):
            file_path = os.path.join(directory, file_name)
            entities = process_resume(file_path)
            match_details = match_criteria(entities)
            if any(match_details.values()):  # Only add if there are matches
                all_matches[file_name] = match_details
    return all_matches

# Directory containing resumes
resumes_directory = 'resumes'

# Process resumes and save results
all_matches = process_all_resumes(resumes_directory)
save_results(all_matches, 'nlp_project/matching_results.txt')
print(f"Results saved to 'nlp_project/matching_results.txt'")


In [ ]:
import os
import spacy
from PyPDF2 import PdfReader
from docx import Document

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Define the criteria for matching resumes
criteria = {
    'skills': ['python', 'tableau', 'sql'],  # Example skills
    'locations': ['south east asia', 'north asia'],  # Example locations
    'experience_years': ['2', '3', '5']  # Example experience years
}

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with open(pdf_path, 'rb') as file:
            reader = PdfReader(file)
            for page in reader.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
    return text

def extract_text_from_docx(docx_path):
    text = ""
    try:
        doc = Document(docx_path)
        for para in doc.paragraphs:
            text += para.text + "\n"
    except Exception as e:
        print(f"Error reading {docx_path}: {e}")
    return text

def extract_entities(text):
    doc = nlp(text)
    entities = {'skills': [], 'locations': [], 'experience_years': []}
    for ent in doc.ents:
        if ent.label_ in ['ORG', 'LOC', 'DATE', 'CARDINAL']:
            if ent.label_ == 'ORG':
                entities['skills'].append(ent.text.lower())
            elif ent.label_ == 'LOC':
                entities['locations'].append(ent.text.lower())
            elif ent.label_ in ['DATE', 'CARDINAL']:
                entities['experience_years'].append(ent.text.lower())
    return entities

def matches_criteria(entities, criteria):
    matches = {
        'skills': [skill for skill in entities['skills'] if skill in criteria['skills']],
        'locations': [loc for loc in entities['locations'] if loc in criteria['locations']],
        'experience_years': [exp for exp in entities['experience_years'] if exp in criteria['experience_years']]
    }
    return matches

def process_resumes(folder_path):
    all_matches = {}
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if file_name.lower().endswith('.pdf'):
            text = extract_text_from_pdf(file_path)
        elif file_name.lower().endswith('.docx'):
            text = extract_text_from_docx(file_path)
        else:
            print(f"Unsupported file type: {file_name}")
            continue
        
        entities = extract_entities(text)
        matches = matches_criteria(entities, criteria)
        if any(matches.values()):  # Only include if there's any match
            all_matches[file_name] = matches
    return all_matches

# Set the path to your resumes directory
resumes_directory = 'resumes'
all_matches = process_resumes(resumes_directory)

# Save the results to a file
def save_results(results, file_path):
    with open(file_path, 'w') as f:
        for file_name, match_details in results.items():
            f.write(f"Resume: {file_name}\n")
            for category, matches in match_details.items():
                f.write(f"{category.capitalize()}: {', '.join(matches)}\n")
            f.write("\n")

save_results(all_matches, 'nlp_project/matching_results.txt')
print("Results saved to 'nlp_project/matching_results.txt'")


In [ ]:
from langdetect import detect
from transformers import pipeline

# Initialize translation pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en")

def detect_and_translate(text):
    lang = detect(text)
    if lang != 'en':
        translated = translator(text, src_lang=lang, tgt_lang='en')
        return translated[0]['translation_text']
    return text

# Example usage
text = "Your resume text here"
processed_text = detect_and_translate(text)


In [ ]:
from langdetect import detect
from transformers import pipeline

# Initialize translation pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en")

def detect_and_translate(text):
    lang = detect(text)
    if lang != 'en':
        translated = translator(text, src_lang=lang, tgt_lang='en')
        return translated[0]['translation_text']
    return text

# Example usage
text = "Your resume text here"
processed_text = detect_and_translate(text)


In [ ]:
from langdetect import detect
from transformers import pipeline

# Initialize translation pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en")

def detect_and_translate(text):
    lang = detect(text)
    if lang != 'en':
        translated = translator(text, src_lang=lang, tgt_lang='en')
        return translated[0]['translation_text']
    return text

# Example usage
text = "Your resume text here"
processed_text = detect_and_translate(text)


In [ ]:
from langdetect import detect
from transformers import pipeline

# Initialize translation pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en")

def detect_and_translate(text):
    lang = detect(text)
    if lang != 'en':
        translated = translator(text, src_lang=lang, tgt_lang='en')
        return translated[0]['translation_text']
    return text

# Example usage
text = "Your resume text here"
processed_text = detect_and_translate(text)


In [ ]:
import torch
print(torch.__version__)



In [ ]:
from langdetect import detect
from transformers import pipeline

# Initialize translation pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en")

def detect_and_translate(text):
    lang = detect(text)
    if lang != 'en':
        translated = translator(text, src_lang=lang, tgt_lang='en')
        return translated[0]['translation_text']
    return text

# Example usage
text = "Your resume text here"
processed_text = detect_and_translate(text)


In [ ]:
from langdetect import detect
from transformers import pipeline

# Initialize translation pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en")

def detect_and_translate(text):
    lang = detect(text)
    if lang != 'en':
        translated = translator(text, src_lang=lang, tgt_lang='en')
        return translated[0]['translation_text']
    return text

# Example usage
text = "Your resume text here"
processed_text = detect_and_translate(text)



In [ ]:
import torch
print(torch.__version__)


In [ ]:
import tensorflow as tf
print(tf.__version__)



In [ ]:
import tensorflow as tf
print(tf.__version__)


In [ ]:
from transformers import pipeline
import torch

# Check if PyTorch is being used
print("Is PyTorch available?", torch.cuda.is_available())
print("PyTorch version:", torch.__version__)

# Initialize translation pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en")


In [ ]:
import torch
print("Is PyTorch available?", torch.cuda.is_available())
print("PyTorch version:", torch.__version__)


In [ ]:
from transformers import pipeline

# Initialize translation pipeline with PyTorch explicitly
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en", framework="pt")

def detect_and_translate(text):
    lang = detect(text)
    if lang != 'en':
        translated = translator(text, src_lang=lang, tgt_lang='en')
        return translated[0]['translation_text']
    return text

# Example usage
text = "Your resume text here"
processed_text = detect_and_translate(text)
print(processed_text)


In [ ]:
python -c "import torch; print(torch.__version__)"


In [ ]:
python -c "import torch; print(torch.cuda.is_available())"


In [ ]:
from transformers import pipeline

# Initialize translation pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en")

def detect_and_translate(text):
    lang = detect(text)  # Ensure you have the `detect` function implemented
    if lang != 'en':
        translated = translator(text, src_lang=lang, tgt_lang='en')
        return translated[0]['translation_text']
    return text

# Example usage
text = "Your resume text here"
processed_text = detect_and_translate(text)
print(processed_text)


In [ ]:
from langdetect import detect
from transformers import pipeline

# Check if PyTorch is available
import torch
print("Is PyTorch available?", torch.cuda.is_available())
print("PyTorch version:", torch.__version__)

# Initialize translation pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en")

def detect_and_translate(text):
    lang = detect(text)
    print(f"Detected language: {lang}")
    if lang != 'en':
        translation = translator(text, src_lang=lang, tgt_lang='en')[0]['translation_text']
        return translation
    return text

# Example usage
sample_text = "Bonjour, comment ça va?"
translated_text = detect_and_translate(sample_text)
print(f"Translated text: {translated_text}")


In [ ]:
import warnings
from urllib3.exceptions import NotOpenSSLWarning
warnings.simplefilter("ignore", NotOpenSSLWarning)

from langdetect import detect
from transformers import pipeline

# Check if PyTorch is available
import torch
print("Is PyTorch available?", torch.cuda.is_available())
print("PyTorch version:", torch.__version__)

# Import tf-keras
import tf_keras as keras

# Initialize translation pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en")

def detect_and_translate(text):
    lang = detect(text)
    print(f"Detected language: {lang}")
    if lang != 'en':
        translation = translator(text, src_lang=lang, tgt_lang='en')[0]['translation_text']
        return translation
    return text

# Example usage
sample_text = "Bonjour, comment ça va?"
translated_text = detect_and_translate(sample_text)
print(f"Translated text: {translated_text}")


In [9]:
from transformers import pipeline
import torch
from langdetect import detect

print("Is PyTorch available?", torch.cuda.is_available())
print("PyTorch version:", torch.__version__)

# Initialize translation pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en")

def detect_and_translate(text):
    lang = detect(text)
    print("Detected language:", lang)
    if lang != "en":
        translated_text = translator(text)[0]['translation_text']
        print("Translated text:", translated_text)
    else:?
        print("Text is already in English:", text)

# Test the function with a sample text
sample_text = "Bonjour, comment ça va?"
detect_and_translate(sample_text)


Is PyTorch available? False
PyTorch version: 2.3.1


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Detected language: fr
Translated text: Hello, how's it going?


In [8]:
print("This is a test output!")


This is a test output!


In [10]:
import fitz  # PyMuPDF
from transformers import pipeline
from langdetect import detect

# Initialize translation pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en")

def detect_and_translate(text):
    lang = detect(text)
    print("Detected language:", lang)
    if lang != "en":
        translated_text = translator(text)[0]['translation_text']
        print("Translated text:", translated_text)
    else:
        print("Text is already in English:", text)

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Extract text from the resume PDF
pdf_path = "resumes/sample_resume.pdf"
resume_text = extract_text_from_pdf(pdf_path)

# Process the extracted text
detect_and_translate(resume_text)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Detected language: en
Text is already in English: INTERNSHIP REPORT ON 
Digital e sales dashboard 
By Sayan Sahu 
VT2024 
Under the guidance of Mr.Rahul Kumar 
About the Project 
Introduction 
The project aims to develop a comprehensive digital sales dashboard 
using Tableau to visualize and analyze sales data, enabling 
stakeholders to make data-driven decisions.
SCOPE
The scope of the project includes data collection, data preprocessing, 
dashboard design, and implementation using Tableau. The dashboard 
provides insights into key sales metrics, trends, and performance 
indicators.
Tools and Technologies: 
-Tableau Desktop:For data visualization and dashboard creation 
-MS Excel:For data collection and preprocessing. 
-Data sheet:Sales data from the company's database. 
PROBLEM STATEMENT 
Problem Statement 1. 
The CFO of an electronics chain is interested in gaining a better 
understanding of sales and 
profits. She has a very specific question of the data 
"What do monthly sales and

In [11]:
import fitz  # PyMuPDF
from transformers import pipeline
from langdetect import detect
import os


In [12]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as pdf_document:
        for page_num in range(len(pdf_document)):
            page = pdf_document.load_page(page_num)
            text += page.get_text()
    return text


In [13]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en")

def detect_and_translate(text):
    lang = detect(text)
    if lang != "en":
        translated_text = translator(text)[0]['translation_text']
        return translated_text
    else:
        return text


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [14]:
def meets_criteria(text, criteria):
    return any(keyword in text.lower() for keyword in criteria)

def shortlist_resumes(resumes_folder, criteria):
    shortlisted_resumes = []
    for filename in os.listdir(resumes_folder):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(resumes_folder, filename)
            resume_text = extract_text_from_pdf(pdf_path)
            translated_text = detect_and_translate(resume_text)
            if meets_criteria(translated_text, criteria):
                shortlisted_resumes.append(filename)
    return shortlisted_resumes



In [15]:
resumes_folder = "resumes"
criteria = ["data analysis", "machine learning"]
shortlisted_files = shortlist_resumes(resumes_folder, criteria)
print("Shortlisted Resumes:", shortlisted_files)


Shortlisted Resumes: []


In [19]:
import os
import pandas as pd
import fitz  # PyMuPDF
from transformers import pipeline
from langdetect import detect
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Initialize translation pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en")

# Define function to read PDF and extract text
def extract_text_from_pdf(pdf_path):
    pdf_document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text += page.get_text()
    pdf_document.close()
    return text

# Define function to detect language and translate text if necessary
def detect_and_translate(text):
    lang = detect(text)
    if lang != "en":
        translated_text = translator(text)[0]['translation_text']
        return translated_text
    return text

# Define function to process resumes and apply criteria
def process_resumes(resumes_folder, criteria):
    resume_texts = []
    resume_files = [f for f in os.listdir(resumes_folder) if f.endswith('.pdf')]
    
    for resume_file in resume_files:
        resume_path = os.path.join(resumes_folder, resume_file)
        text = extract_text_from_pdf(resume_path)
        translated_text = detect_and_translate(text)
        resume_texts.append(translated_text)

    # Apply criteria
    vectorizer = CountVectorizer().fit_transform(resume_texts)
    vectors = vectorizer.toarray()
    cosine_matrix = cosine_similarity(vectors)
    
    # Example of filtering resumes based on criteria
    # Here you would implement your specific criteria for filtering resumes
    # For simplicity, this example assumes you want to select the first 5 resumes
    filtered_indices = cosine_matrix[0].argsort()[-5:][::-1]
    filtered_resumes = [resume_files[i] for i in filtered_indices]
    
    return filtered_resumes

# Define criteria for shortlisting (this should be customized based on your needs)
criteria = {}  # Example criteria dictionary, modify as needed

# Run the process
resumes_folder = 'resumes'
shortlisted_resumes = process_resumes(resumes_folder, criteria)
print("Shortlisted Resumes:", shortlisted_resumes)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Shortlisted Resumes: ['sample_resume.pdf']


In [20]:
import os
import pandas as pd
import fitz  # PyMuPDF
from transformers import pipeline
from langdetect import detect
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Initialize translation pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en")

# Define function to read PDF and extract text
def extract_text_from_pdf(pdf_path):
    pdf_document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text += page.get_text()
    pdf_document.close()
    return text

# Define function to detect language and translate text if necessary
def detect_and_translate(text):
    lang = detect(text)
    if lang != "en":
        translated_text = translator(text)[0]['translation_text']
        return translated_text
    return text

# Define function to process resumes and apply criteria
def process_resumes(resumes_folder, criteria):
    resume_texts = []
    resume_files = [f for f in os.listdir(resumes_folder) if f.endswith('.pdf')]
    
    for resume_file in resume_files:
        resume_path = os.path.join(resumes_folder, resume_file)
        text = extract_text_from_pdf(resume_path)
        translated_text = detect_and_translate(text)
        resume_texts.append(translated_text)

    # Apply criteria
    vectorizer = CountVectorizer().fit_transform(resume_texts)
    vectors = vectorizer.toarray()
    cosine_matrix = cosine_similarity(vectors)
    
    # Example of filtering resumes based on criteria
    # Here you would implement your specific criteria for filtering resumes
    # For simplicity, this example assumes you want to select the first 5 resumes
    filtered_indices = cosine_matrix[0].argsort()[-5:][::-1]
    filtered_resumes = [resume_files[i] for i in filtered_indices]
    
    return filtered_resumes

# Define criteria for shortlisting (this should be customized based on your needs)
criteria = {}  # Example criteria dictionary, modify as needed

# Run the process
resumes_folder = 'resumes'
shortlisted_resumes = process_resumes(resumes_folder, criteria)
print("Shortlisted Resumes:", shortlisted_resumes)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Shortlisted Resumes: ['sample_resume.pdf']


In [21]:
import os
import fitz  # PyMuPDF
from transformers import pipeline
from langdetect import detect
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import torch

# Initialize translation pipeline with device argument for GPU usage
device = 0 if torch.cuda.is_available() else -1
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en", device=device)

# Define function to read PDF and extract text
def extract_text_from_pdf(pdf_path):
    pdf_document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text += page.get_text()
    pdf_document.close()
    return text

# Define function to detect language and translate text if necessary
def detect_and_translate(text):
    lang = detect(text)
    if lang != "en":
        translated_text = translator(text)[0]['translation_text']
        return translated_text
    return text

# Define function to process resumes and apply criteria
def process_resumes(resumes_folder, criteria):
    resume_texts = []
    resume_files = [f for f in os.listdir(resumes_folder) if f.endswith('.pdf')]
    
    for resume_file in resume_files:
        resume_path = os.path.join(resumes_folder, resume_file)
        text = extract_text_from_pdf(resume_path)
        translated_text = detect_and_translate(text)
        resume_texts.append(translated_text)

    # Apply criteria
    vectorizer = CountVectorizer().fit_transform(resume_texts)
    vectors = vectorizer.toarray()
    cosine_matrix = cosine_similarity(vectors)
    
    # Example of filtering resumes based on criteria
    # Here you would implement your specific criteria for filtering resumes
    # For simplicity, this example assumes you want to select the first 5 resumes
    filtered_indices = cosine_matrix[0].argsort()[-5:][::-1]
    filtered_resumes = [resume_files[i] for i in filtered_indices]
    
    return filtered_resumes

# Define criteria for shortlisting (this should be customized based on your needs)
criteria = {}  # Example criteria dictionary, modify as needed

# Run the process
resumes_folder = 'resumes'
shortlisted_resumes = process_resumes(resumes_folder, criteria)
print("Shortlisted Resumes:", shortlisted_resumes)


Shortlisted Resumes: ['sample_resume.pdf']


In [22]:
import os
import fitz  # PyMuPDF
from transformers import pipeline
from langdetect import detect
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Initialize translation pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en")

# Define function to read PDF and extract text
def extract_text_from_pdf(pdf_path):
    pdf_document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text += page.get_text()
    pdf_document.close()
    return text

# Define function to detect language and translate text if necessary
def detect_and_translate(text):
    lang = detect(text)
    print(f"Detected language: {lang}")  # Debug: Print detected language
    if lang != "en":
        translated_text = translator(text)[0]['translation_text']
        print(f"Translated text: {translated_text}")  # Debug: Print translated text
        return translated_text
    return text

# Define function to process resumes and apply criteria
def process_resumes(resumes_folder, criteria):
    resume_texts = []
    resume_files = [f for f in os.listdir(resumes_folder) if f.endswith('.pdf')]
    
    for resume_file in resume_files:
        resume_path = os.path.join(resumes_folder, resume_file)
        text = extract_text_from_pdf(resume_path)
        translated_text = detect_and_translate(text)
        resume_texts.append(translated_text)

    # Apply criteria
    vectorizer = CountVectorizer().fit_transform(resume_texts)
    vectors = vectorizer.toarray()
    cosine_matrix = cosine_similarity(vectors)
    
    # Example of filtering resumes based on criteria
    # Here you would implement your specific criteria for filtering resumes
    # For simplicity, this example assumes you want to select the first 5 resumes
    filtered_indices = cosine_matrix[0].argsort()[-5:][::-1]
    filtered_resumes = [resume_files[i] for i in filtered_indices]
    
    return filtered_resumes

# Define criteria for shortlisting (this should be customized based on your needs)
criteria = {}  # Example criteria dictionary, modify as needed

# Run the process
resumes_folder = 'resumes'
shortlisted_resumes = process_resumes(resumes_folder, criteria)
print("Shortlisted Resumes:", shortlisted_resumes)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Detected language: en
Shortlisted Resumes: ['sample_resume.pdf']


In [23]:
import os
import fitz  # PyMuPDF
from transformers import pipeline
from langdetect import detect
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Initialize translation pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en")

# Define function to read PDF and extract text
def extract_text_from_pdf(pdf_path):
    pdf_document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text += page.get_text()
    pdf_document.close()
    return text

# Define function to detect language and translate text if necessary
def detect_and_translate(text):
    lang = detect(text)
    if lang != "en":
        translated_text = translator(text)[0]['translation_text']
        return translated_text, lang
    return text, lang

# Define function to process resumes and apply criteria
def process_resumes(resumes_folder, criteria):
    resume_texts = []
    resume_files = [f for f in os.listdir(resumes_folder) if f.endswith('.pdf')]
    resume_info = []

    for resume_file in resume_files:
        resume_path = os.path.join(resumes_folder, resume_file)
        text = extract_text_from_pdf(resume_path)
        translated_text, lang = detect_and_translate(text)
        resume_texts.append(translated_text)
        resume_info.append((resume_file, lang))

    # Apply criteria
    vectorizer = CountVectorizer().fit_transform(resume_texts)
    vectors = vectorizer.toarray()
    cosine_matrix = cosine_similarity(vectors)
    
    # Example of filtering resumes based on criteria
    filtered_indices = cosine_matrix[0].argsort()[-5:][::-1]
    filtered_resumes = [resume_info[i] for i in filtered_indices]
    
    return filtered_resumes

# Define criteria for shortlisting (this should be customized based on your needs)
criteria = {}  # Example criteria dictionary, modify as needed

# Run the process
resumes_folder = 'resumes'
shortlisted_resumes = process_resumes(resumes_folder, criteria)

# Save results to a text file
with open("shortlisted_resumes.txt", "w") as file:
    for resume, lang in shortlisted_resumes:
        file.write(f"{resume}: Detected language - {lang}\n")

print("Shortlisted Resumes saved to 'shortlisted_resumes.txt'")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Shortlisted Resumes saved to 'shortlisted_resumes.txt'


In [24]:
import os
import fitz  # PyMuPDF
from transformers import pipeline
from langdetect import detect
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from docx import Document

# Initialize translation pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en")

# Define function to read PDF and extract text
def extract_text_from_pdf(pdf_path):
    pdf_document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text += page.get_text()
    pdf_document.close()
    return text

# Define function to detect language and translate text if necessary
def detect_and_translate(text):
    lang = detect(text)
    print(f"Detected language: {lang}")  # Debug: Print detected language
    if lang != "en":
        translated_text = translator(text)[0]['translation_text']
        print(f"Translated text: {translated_text}")  # Debug: Print translated text
        return translated_text, lang
    return text, lang

# Define function to process resumes and apply criteria
def process_resumes(resumes_folder, criteria):
    resume_texts = []
    resume_files = [f for f in os.listdir(resumes_folder) if f.endswith('.pdf')]
    resume_info = []

    for resume_file in resume_files:
        resume_path = os.path.join(resumes_folder, resume_file)
        text = extract_text_from_pdf(resume_path)
        translated_text, lang = detect_and_translate(text)
        resume_texts.append(translated_text)
        resume_info.append((resume_file, lang, translated_text))

    # Apply criteria
    vectorizer = CountVectorizer().fit_transform(resume_texts)
    vectors = vectorizer.toarray()
    cosine_matrix = cosine_similarity(vectors)
    
    # Example of filtering resumes based on criteria
    filtered_indices = cosine_matrix[0].argsort()[-5:][::-1]
    filtered_resumes = [resume_info[i] for i in filtered_indices]
    
    return filtered_resumes

# Define criteria for shortlisting (this should be customized based on your needs)
criteria = {}  # Example criteria dictionary, modify as needed

# Run the process
resumes_folder = 'resumes'
shortlisted_resumes = process_resumes(resumes_folder, criteria)

# Save results to a Word document
doc = Document()
doc.add_heading('Shortlisted Resumes', 0)

for resume, lang, text in shortlisted_resumes:
    doc.add_paragraph(f"Resume: {resume}")
    doc.add_paragraph(f"Detected language: {lang}")
    doc.add_paragraph(f"Translated text: {text}")
    doc.add_paragraph()  # Add a blank line between entries

doc.save("shortlisted_resumes.docx")

print("Shortlisted Resumes saved to 'shortlisted_resumes.docx'")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Detected language: en
Shortlisted Resumes saved to 'shortlisted_resumes.docx'


In [25]:
import os
import fitz  # PyMuPDF
from transformers import pipeline
from langdetect import detect
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from docx import Document

# Initialize translation pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en")

# Define function to read PDF and extract text
def extract_text_from_pdf(pdf_path):
    pdf_document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text += page.get_text()
    pdf_document.close()
    return text

# Define function to detect language and translate text if necessary
def detect_and_translate(text):
    lang = detect(text)
    print(f"Detected language: {lang}")  # Debug: Print detected language
    if lang != "en":
        translated_text = translator(text)[0]['translation_text']
        print(f"Translated text: {translated_text}")  # Debug: Print translated text
        return translated_text, lang
    return text, lang

# Define function to clean up text
def clean_text(text):
    # Replace special characters and extra spaces
    text = text.replace(' ', '\n')  # Replace special line breaks
    text = text.replace('\n\n', '\n')  # Remove extra line breaks
    text = text.strip()  # Remove leading and trailing whitespace
    return text

# Define function to process resumes and apply criteria
def process_resumes(resumes_folder, criteria):
    resume_texts = []
    resume_files = [f for f in os.listdir(resumes_folder) if f.endswith('.pdf')]
    resume_info = []

    for resume_file in resume_files:
        resume_path = os.path.join(resumes_folder, resume_file)
        text = extract_text_from_pdf(resume_path)
        translated_text, lang = detect_and_translate(text)
        cleaned_text = clean_text(translated_text)
        resume_texts.append(cleaned_text)
        resume_info.append((resume_file, lang, cleaned_text))

    # Apply criteria
    vectorizer = CountVectorizer().fit_transform(resume_texts)
    vectors = vectorizer.toarray()
    cosine_matrix = cosine_similarity(vectors)
    
    # Example of filtering resumes based on criteria
    filtered_indices = cosine_matrix[0].argsort()[-5:][::-1]
    filtered_resumes = [resume_info[i] for i in filtered_indices]
    
    return filtered_resumes

# Define criteria for shortlisting (this should be customized based on your needs)
criteria = {}  # Example criteria dictionary, modify as needed

# Run the process
resumes_folder = 'resumes'
shortlisted_resumes = process_resumes(resumes_folder, criteria)

# Save results to a Word document
doc = Document()
doc.add_heading('Shortlisted Resumes', 0)

for resume, lang, text in shortlisted_resumes:
    doc.add_paragraph(f"Resume: {resume}")
    doc.add_paragraph(f"Detected language: {lang}")
    doc.add_paragraph("Translated text:")
    doc.add_paragraph(text)
    doc.add_paragraph()  # Add a blank line between entries

doc.save("shortlisted_resumes.docx")

print("Shortlisted Resumes saved to 'shortlisted_resumes.docx'")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Detected language: en
Shortlisted Resumes saved to 'shortlisted_resumes.docx'
